In [16]:
# What version of Python do you have?
import sys

import dask.dataframe as dd
import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
import platform
from typing import List

print(f"Running on machine: {platform.uname()}")

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Running on machine: uname_result(system='Darwin', node='dkfepaha.local', release='22.6.0', version='Darwin Kernel Version 22.6.0: Wed Jul  5 22:22:05 PDT 2023; root:xnu-8796.141.3~6/RELEASE_ARM64_T6000', machine='arm64')
Tensor Flow Version: 2.5.0
Keras Version: 2.5.0

Python 3.9.15 | packaged by conda-forge | (main, Nov 22 2022, 08:52:10) 
[Clang 14.0.6 ]
Pandas 1.4.4
Scikit-Learn 1.1.2
GPU is available


In [17]:
from dask.distributed import Client
client = Client()
client

/Users/dkFePaHa/miniconda3/envs/tensorflow/lib/python3.9/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 62814 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:62814/status,
Dashboard: http://127.0.0.1:62814/status,Workers: 5
Total threads: 10,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:62815,Workers: 5
Dashboard: http://127.0.0.1:62814/status,Total threads: 10
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:62836,Total threads: 2
Dashboard: http://127.0.0.1:62841/status,Memory: 6.40 GiB
Nanny: tcp://127.0.0.1:62819,


In [18]:
df = dd.read_parquet(f"archive/otto-recsys-train.jsonl-parquet")
test_df = dd.read_parquet(f"archive/otto-recsys-test.jsonl-parquet")
normalized = df
df.compute()

,aid,ts,type,session
0,1517085,1659304800025,clicks,0
1,1563459,1659304904511,clicks,0
2,1309446,1659367439426,clicks,0
3,16246,1659367719997,clicks,0
4,1781822,1659367871344,clicks,0
...,...,...,...,...
960804,1737908,1661723987073,clicks,12899776
960805,384045,1661723976974,clicks,12899777
960806,384045,1661723986800,clicks,12899777
960807,561560,1661723983611,clicks,12899778


In [19]:
mask = normalized['type'] == "orders"

orders = normalized[mask]


sessions_with_orders = len(orders["session"].unique())
session_number = len(normalized["session"].unique())
vocabulary = normalized["aid"].unique().compute().tolist()
unique_aids = len(vocabulary)
percentage_with_purchase = sessions_with_orders / session_number * 100

total_orders = len(orders)
average_order_per_buying_session = total_orders / sessions_with_orders

normalized[mask].groupby("session")

MINIMUM_ORDERS = 2

sessions_with_more_than_k_orders = orders.map_partitions(lambda df: df.groupby("session").filter(lambda x: len(x) > MINIMUM_ORDERS))
sessions_with_more_than_k_orders.compute()

num_sessions_with_more_than_k_orders = len(sessions_with_more_than_k_orders["session"].unique())

print(f"Number of unique aids: {unique_aids}")

print(f"Total number of orders: {total_orders}")
print(f"Average number of orders per buying session: {average_order_per_buying_session}")

print(f"Number of sessions with more than 3 orders: {num_sessions_with_more_than_k_orders}")

print(f"Number of sessions with orders: {sessions_with_orders}")
print(f"Percentage of sessions with orders: {percentage_with_purchase:.2f}%")


Number of unique aids: 1855603
Total number of orders: 5098951
Average number of orders per buying session: 3.1352344961502467
Number of sessions with more than 3 orders: 602169
Number of sessions with orders: 1626338
Percentage of sessions with orders: 12.61%


In [20]:
display(orders["aid"].value_counts().nlargest(10).compute())
display(orders["aid"].value_counts().nsmallest(10).compute())

231487     4485
166037     3824
1733943    3042
1445562    2998
1022566    2788
801774     2710
1629608    2628
756588     2593
332654     2436
1603001    2353
Name: aid, dtype: int64

1444720    1
1413905    1
1337241    1
1341311    1
1340657    1
1341294    1
1360650    1
1357683    1
1360352    1
1449712    1
Name: aid, dtype: int64

In [21]:
pd.set_option('display.max_rows', 30)
filtered = sessions_with_more_than_k_orders
filtered = filtered[["aid","session"]]

filtered.compute()

,aid,session
8,305831,0
9,461689,0
245,1199474,0
246,543308,0
360,357461,3
...,...,...
959388,465366,12899355
960085,1488793,12899525
960086,1599360,12899525
960087,996393,12899525


In [22]:
orders_by_session: List[List[int]] = filtered.compute().groupby("session").agg(list)["aid"].values.tolist()
print(len(orders_by_session))
orders_by_session

602169


[[305831, 461689, 1199474, 543308],
 [357461, 1343406, 1425967, 1018433, 54857],
 [887719, 1742191, 618376, 1689065, 1757034, 1670763],
 [571631,
  588838,
  275825,
  326526,
  1054943,
  1664305,
  1815907,
  1077924,
  1051675,
  816119,
  796017,
  284816,
  125127,
  1566388,
  772148],
 [1213426, 1006109, 165455, 719255, 383546, 228850, 383546],
 [806996, 1534972, 1415436],
 [1708372, 1758952, 51179],
 [1104781, 1404100, 1404100, 573705, 1162085, 1182605, 1148596],
 [832382, 166037, 915392, 1240180, 1082079, 1500844, 726640, 205516],
 [1660020,
  934266,
  1839606,
  1470083,
  1639625,
  1097946,
  565802,
  576646,
  509070,
  1384883,
  1438538,
  409620,
  409620,
  282621,
  1018338],
 [90173, 1086435, 695183, 545295, 701694, 1341, 235666, 143617, 1679916],
 [223422, 260934, 987525, 483149],
 [1041839, 639869, 636101],
 [1308823, 399315, 453736],
 [930005,
  801774,
  946991,
  1112168,
  1112168,
  1433523,
  519672,
  365522,
  709156,
  519672,
  1357072,
  147641],
 [166

In [23]:
from split import train_test_split
print(f"orders_by_session has {len(orders_by_session)} sessions")
test_size = 3000
assert test_size < len(orders_by_session), "test_size must be smaller than the number of sessions"

train, test, valid = train_test_split(orders_by_session, test_size=test_size)

orders_by_session has 602169 sessions


In [24]:
from gensim.models.callbacks import CallbackAny2Vec


class LossLogger(CallbackAny2Vec):
    """Output loss at each epoch"""

    def __init__(self):
        self.epoch = 1
        self.losses = []

    def on_epoch_begin(self, model):
        print(f"Epoch: {self.epoch}", end="\t")

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        last_loss = self.losses[self.epoch - 2] if len(self.losses) > 0 else None
        delta = (
            loss - last_loss
            if last_loss is not None
            else loss
        )
        self.losses.append(loss)
        print(f"  Loss: {delta}")
        self.epoch += 1

In [25]:
from gensim.models import Word2Vec

from evaluation import recall_at_k, mrr_at_k, RandomEmbeddings
loss_logger = LossLogger()

print(f"Training Word2Vec model on {len(train)} sessions")
model = Word2Vec(
    train,
    min_count=1,
    sg=1,
    workers=8,
    epochs=200,
    compute_loss=True,
    callbacks=[loss_logger],
)

print("Saving model")
model.save("word2vec.model")

embeddings = model.wv  # "word vectors"
# sims = embeddings.most_similar('computer', topn=10)

K = 20

random_embeddings = RandomEmbeddings(vocabulary)

print(f"Evaluating model with 'recall@{K}'")
recall_score = recall_at_k(test, embeddings, k=K)
recall_score_random = recall_at_k(test, random_embeddings, k=K)
print(recall_score)
print(f"Random score: {recall_score_random}")

print(f"Evaluating model with 'mrr@{K}'")
mrr_score = mrr_at_k(test, embeddings, k=K)
mrr_score_random = mrr_at_k(test, random_embeddings, k=K)
print(mrr_score)
print(f"Random mrr score: {mrr_score_random}")




Training Word2Vec model on 602169 sessions
Epoch: 1	  Loss: 5666268.5
Epoch: 2	  Loss: 3749840.5
Epoch: 3	  Loss: 2644489.0
Epoch: 4	  Loss: 2178509.0
Epoch: 5	  Loss: 1860367.0
Epoch: 6	  Loss: 1552936.0
Epoch: 7	  Loss: 1511042.0
Epoch: 8	  Loss: 1363748.0
Epoch: 9	  Loss: 1304642.0
Epoch: 10	  Loss: 1256292.0
Epoch: 11	  Loss: 1156488.0
Epoch: 12	  Loss: 1094486.0
Epoch: 13	  Loss: 1017112.0
Epoch: 14	  Loss: 940084.0
Epoch: 15	  Loss: 842462.0
Epoch: 16	  Loss: 802390.0
Epoch: 17	  Loss: 740044.0
Epoch: 18	  Loss: 674216.0
Epoch: 19	  Loss: 628532.0
Epoch: 20	  Loss: 588270.0
Epoch: 21	  Loss: 559348.0
Epoch: 22	  Loss: 529444.0
Epoch: 23	  Loss: 521818.0
Epoch: 24	  Loss: 433592.0
Epoch: 25	  Loss: 256568.0
Epoch: 26	  Loss: 250704.0
Epoch: 27	  Loss: 248388.0
Epoch: 28	  Loss: 240260.0
Epoch: 29	  Loss: 244884.0
Epoch: 30	  Loss: 239880.0
Epoch: 31	  Loss: 235692.0
Epoch: 32	  Loss: 233032.0
Epoch: 33	  Loss: 234552.0
Epoch: 34	  Loss: 234544.0
Epoch: 35	  Loss: 230428.0
Epoch: 3

2023-09-29 06:59:00,848 - tornado.application - ERROR - Exception in callback <bound method SystemMonitor.update of <SystemMonitor: cpu: 17 memory: 249 MB fds: 334>>
Traceback (most recent call last):
  File "/Users/dkFePaHa/miniconda3/envs/tensorflow/lib/python3.9/site-packages/distributed/compatibility.py", line 161, in _run
    val = self.callback()
  File "/Users/dkFePaHa/miniconda3/envs/tensorflow/lib/python3.9/site-packages/distributed/system_monitor.py", line 133, in update
    net_ioc = psutil.net_io_counters()
  File "/Users/dkFePaHa/miniconda3/envs/tensorflow/lib/python3.9/site-packages/psutil/__init__.py", line 2119, in net_io_counters
    rawdict = _psplatform.net_io_counters()
OSError: [Errno 12] Cannot allocate memory
